In [11]:
import pandas as pd
import requests as rq
import utils.constants as constants
import streamlit as st

url1 = constants.RDW_KENTEKEN_URL
url2 = constants.RDW_BRANDSTOF_API_URL

json_kenteken = rq.get(url1, 
headers={"X-App-Token": st.secrets['RDW_API_KEY']}, params={"$limit": 300_0000}).json()

json_brandstof = rq.get(url2, 
headers={"X-App-Token": st.secrets['RDW_API_KEY']}, params={"$limit": 300_0000}).json()

df1 = pd.DataFrame.from_dict(json_kenteken)
df2 = pd.DataFrame.from_dict(json_brandstof)

df = pd.merge(df1, df2, on='kenteken')

print(len(df))
# df1['kenteken'] = df1['kenteken'].astype(str)
# df2['kenteken'] = df2['kenteken'].astype(str)

# df1.info()
# print(df1)
# print(df2)


# response = rq.get(constants.OCM_API_URL, 
# headers={"X-API-Key": st.secrets['OCM_API_KEY']}).json()
# pd.DataFrame.from_dict(response).head(5)

636227


In [74]:
def fetchOCMData(isFilters=False):
    response = rq.get("https://api.openchargemap.io/v3/poi/?output=json&countrycode=NL&verbose=false&includecomments=false", 
    headers={"X-API-Key": st.secrets['OCM_API_KEY']}).json()
    df = pd.json_normalize(response)
    fitlers = pd.json_normalize(response, record_path=["Connections"])
    listOfCols = ['IsRecentlyVerified', 'UsageType.Title', 'AddressInfo.Latitude', 'AddressInfo.Longitude', 'Connections', 'NumberOfPoints', 'StatusTypeID']
    df['Connections'] = df['Connections'].apply(lambda x: [y['ConnectionType']['Title'] for y in x])
    df.drop(columns=df.columns[~df.columns.isin(listOfCols)].values, axis=1, inplace=True)
    df = df[df['IsRecentlyVerified']]
    if (isFilters):
        return fitlers['ConnectionType.Title'].unique()
    return df

df = fetchOCMData()

,IsRecentlyVerified,Connections,NumberOfPoints,StatusTypeID,UsageType.Title,AddressInfo.Latitude,AddressInfo.Longitude
0,True,"[CHAdeMO, CCS (Type 2)]",2,50,Public - Membership Required,52.154843,6.201008
1,True,[Type 2 (Socket Only)],2,50,Public - Membership Required,52.220917,4.531350
2,True,[Type 2 (Socket Only)],1,50,Public - Membership Required,51.411700,6.167800
3,True,[Type 2 (Socket Only)],1,50,Public,52.042242,5.665901
4,True,[Type 2 (Socket Only)],2,50,"Private - For Staff, Visitors or Customers",52.399358,6.783781


In [43]:
#duplicates op basis van nummerbord droppen
df = df.drop_duplicates(subset = ["kenteken"], keep = "last")

#check
df[df.duplicated(subset = ["kenteken"], keep = "last" )].sort_values("kenteken")

,kenteken,voertuigsoort,merk,handelsbenaming,vervaldatum_apk,datum_tenaamstelling,bruto_bpm,inrichting,aantal_zitplaatsen,eerste_kleur,...,max_vermogen_60_minuten,max_vermogen_15_minuten,elektrisch_verbruik_extern_opladen_wltp,actie_radius_extern_opladen_wltp,actie_radius_extern_opladen_stad_wltp,co2_uitstoot_gewogen,emis_deeltjes_type1_wltp,emis_co2_gewogen_gecombineerd_wltp,brandstof_verbruik_gewogen_gecombineerd_wltp,uitstoot_deeltjes_zwaar


In [46]:
# alleen noodzakelijke colomns bewaren
clist = ["voertuigsoort", "merk", "handelsbenaming", "datum_tenaamstelling", "brandstof_omschrijving"]

df_clean = df.drop(columns=df.columns[~df.columns.isin(clist)].values, axis=1)

#datum_tenaamstelling to datetime
df_clean["datum_tenaamstelling"] = pd.to_datetime(df_clean["datum_tenaamstelling"])
# drop onbekende datum tenaamstelling
df_clean =  df_clean.dropna(subset = ["datum_tenaamstelling"])

df_clean.head()

,voertuigsoort,merk,handelsbenaming,datum_tenaamstelling,brandstof_omschrijving
0,Personenauto,OPEL,CORSA,2020-01-21,Benzine
1,Bromfiets,LA SOURIS,SOURINI,2020-07-10,Benzine
2,Personenauto,RENAULT,MEGANE SCENIC,2016-06-03,Benzine
3,Personenauto,VOLKSWAGEN,POLO,2020-03-17,Benzine
4,Bedrijfsauto,TOYOTA,LAND CRUISER V8,2020-10-13,Diesel


In [49]:
print(df_clean.isna().sum())
df_clean["datum_tenaamstelling"].min()

df_clean.shape

df_clean.to_csv("clean 621517")





voertuigsoort              0
merk                       0
handelsbenaming           87
datum_tenaamstelling       0
brandstof_omschrijving     0
dtype: int64
